In [1]:
#Librerias 
import pandas as pd
import numpy as np
import os
import re

In [ ]:
# --- Constantes y Parámetros ---
# Establece el formato de fecha esperado en los datos (mes/día/año)
EXPECTED_DATE_FORMAT = '%m/%d/%y' 

# --- Rutas de Archivos ---
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir) 

# Ruta al archivo RAW
raw_data_path = os.path.join(project_root, 'data', 'raw', 'NetflixViewingHistory.csv')
# Ruta opcional para guardar el archivo limpiado intermedio
interim_data_path = os.path.join(project_root, 'data', 'interim', 'netflix_history_cleaned.csv')

print(f"Ruta al archivo RAW: {raw_data_path}")
print(f"Ruta al archivo Intermedio (opcional): {interim_data_path}")

Ruta al archivo RAW: h:\git\Analisis-de_consumo_Netflix\Analisis-de_consumo_Netflix\data\raw\NetflixViewingHistory.csv
Ruta al archivo Intermedio (opcional): h:\git\Analisis-de_consumo_Netflix\Analisis-de_consumo_Netflix\data\interim\netflix_history_cleaned.csv


In [3]:
# --- Cargar Datos Crudos ---
print(f"Cargando datos desde: {raw_data_path}")
try:
    df_raw = pd.read_csv(raw_data_path)
    print(f"Datos crudos cargados: {df_raw.shape[0]} filas, {df_raw.shape[1]} columnas.")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en {raw_data_path}. Verifica la ruta.")
    raise SystemExit("Archivo no encontrado.")
except Exception as e:
    print(f"Error al cargar el archivo CSV crudo: {e}")
    raise SystemExit("Error cargando CSV.")

# Crear una copia para trabajar y no modificar el original cargado
df_clean = df_raw.copy()

print("\n--- Primeras filas de los datos crudos ---")
display(df_clean.head())

Cargando datos desde: h:\git\Analisis-de_consumo_Netflix\Analisis-de_consumo_Netflix\data\raw\NetflixViewingHistory.csv
Datos crudos cargados: 1875 filas, 2 columnas.

--- Primeras filas de los datos crudos ---


,Title,Date
0,La casa de papel: Parte 5: La teoría de la ele...,5/5/25
1,La casa de papel: Parte 5: Ciencia ilusionada,5/5/25
2,La casa de papel: Parte 5: Válvulas de escape,5/5/25
3,La casa de papel: Parte 5: Vivir muchas vidas,5/5/25
4,La casa de papel: Parte 5: Tu sitio en el cielo,5/5/25


In [ ]:
# --- Inspección Inicial ---
print("\n--- Información General (Tipos de Dato y Nulos Iniciales) ---")
df_clean.info()

print("\n--- Conteo de Valores Nulos por Columna ---")
print(df_clean.isnull().sum())

# Verificar si las columnas esperadas existen
required_columns = ['Title', 'Date']
if not all(col in df_clean.columns for col in required_columns):
    print(f"\n¡Advertencia! Faltan columnas requeridas. Se esperaban: {required_columns}. Columnas encontradas: {df_clean.columns.tolist()}")
  


--- Información General (Tipos de Dato y Nulos Iniciales) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1875 entries, 0 to 1874
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   1874 non-null   object
 1   Date    1875 non-null   object
dtypes: object(2)
memory usage: 29.4+ KB

--- Conteo de Valores Nulos por Columna ---
Title    1
Date     0
dtype: int64


In [5]:
# --- Limpieza Básica ---

print("\n--- Limpiando filas con Nulos en 'Title' o 'Date' ---")
rows_before = len(df_clean)
df_clean.dropna(subset=['Title', 'Date'], inplace=True)
rows_after = len(df_clean)
print(f"Filas eliminadas por nulos en Title/Date: {rows_before - rows_after}")
print(f"Filas restantes: {rows_after}")

print(f"\n--- Convirtiendo 'Date' a formato datetime (Esperado: {EXPECTED_DATE_FORMAT}) ---")
try:
    # Intentar convertir usando el formato especificado
    df_clean['Date'] = pd.to_datetime(df_clean['Date'], format=EXPECTED_DATE_FORMAT, errors='coerce')
    
    # Verificar cuántas fechas fallaron en la conversión (se vuelven NaT)
    nat_count = df_clean['Date'].isnull().sum()
    if nat_count > 0:
        print(f"¡Advertencia! {nat_count} fechas no pudieron ser parseadas con el formato '{EXPECTED_DATE_FORMAT}' y se convirtieron a NaT (Not a Time).")
        # Opcional: Mostrar algunas filas con fechas problemáticas
        # display(df_raw.loc[df_clean['Date'].isnull()]) # Muestra las filas originales donde la fecha falló
        
        # Eliminar filas donde la fecha es NaT
        rows_before_nat_drop = len(df_clean)
        df_clean.dropna(subset=['Date'], inplace=True)
        print(f"Filas eliminadas debido a fechas inválidas (NaT): {rows_before_nat_drop - len(df_clean)}")
        print(f"Filas restantes después de limpieza de fechas: {len(df_clean)}")
    else:
        print("Todas las fechas fueron convertidas exitosamente.")
        
except ValueError as ve:
    print(f"\n¡Error! El formato de fecha '{EXPECTED_DATE_FORMAT}' parece incorrecto para tus datos.")
    print(f"Error específico: {ve}")
    print("Revisa la constante EXPECTED_DATE_FORMAT en la Celda 1 y el contenido de la columna 'Date' en tu CSV.")
    # Podrías querer detener la ejecución aquí o continuar sin la conversión de fecha correcta.
except Exception as e:
    print(f"Error inesperado durante la conversión de fechas: {e}")


print("\n--- Información después de limpieza básica ---")
df_clean.info()
print("\n--- Nulos después de limpieza básica ---")
print(df_clean.isnull().sum())


--- Limpiando filas con Nulos en 'Title' o 'Date' ---
Filas eliminadas por nulos en Title/Date: 1
Filas restantes: 1874

--- Convirtiendo 'Date' a formato datetime (Esperado: %m/%d/%y) ---
Todas las fechas fueron convertidas exitosamente.

--- Información después de limpieza básica ---
<class 'pandas.core.frame.DataFrame'>
Index: 1874 entries, 0 to 1874
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Title   1874 non-null   object        
 1   Date    1874 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 43.9+ KB

--- Nulos después de limpieza básica ---
Title    0
Date     0
dtype: int64
